<a href="https://colab.research.google.com/github/muhammadalfateh6/Muhammad_Alfateh_Work/blob/main/Week_4_Saving_weights%26model_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programming Assignment

## Saving and loading models, with application to the EuroSat dataset

### Instructions

In this notebook, you will create a neural network that classifies land uses and land covers from satellite imagery. You will save your model using Tensorflow's callbacks and reload it later. You will also load in a pre-trained neural network classifier and compare performance with it. 

Some code cells are provided for you in the notebook. You should avoid editing provided code, and make sure to execute the cells in order to avoid unexpected errors. Some cells begin with the line: 

`#### GRADED CELL ####`

Don't move or edit this first line - this is what the automatic grader looks for to recognise graded cells. These cells require you to write your own code to complete them, and are automatically graded when you submit the notebook. Don't edit the function name or signature provided in these cells, otherwise the automatic grader might not function properly. Inside these graded cells, you can use any functions or classes that are imported below, but make sure you don't use any variables that are outside the scope of the function.

### How to submit

Complete all the tasks you are asked for in the worksheet. When you have finished and are happy with your code, press the **Submit Assignment** button at the top of this notebook.

### Let's get started!

We'll start running some imports, and loading the dataset. Do not edit the existing imports in the following cell. If you would like to make further Tensorflow imports, you should add them here.

In [ ]:
#### PACKAGE IMPORTS ####

# Run this cell first to import all required packages. Do not make any imports elsewhere in the notebook

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import numpy as np
import pandas as pd
from tensorflow.train import latest_checkpoint
# If you would like to make further imports from tensorflow, add them here



#### The EuroSAT dataset

In this assignment, you will use the [EuroSAT dataset](https://github.com/phelber/EuroSAT). It consists of 27000 labelled Sentinel-2 satellite images of different land uses: residential, industrial, highway, river, forest, pasture, herbaceous vegetation, annual crop, permanent crop and sea/lake. For a reference, see the following papers:
- Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification. Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.
- Introducing EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification. Patrick Helber, Benjamin Bischke, Andreas Dengel. 2018 IEEE International Geoscience and Remote Sensing Symposium, 2018.

Your goal is to construct a neural network that classifies a satellite image into one of these 10 classes, as well as applying some of the saving and loading techniques you have learned in the previous sessions.

#### Import the data

The dataset you will train your model on is a subset of the total data, with 4000 training images and 1000 testing images, with roughly equal numbers of each class. 
The train and test datasets required for this project can be downloaded from the following links:

`x_train`: https://drive.google.com/open?id=1cUaIEd9-MLJHFGjLz5QziNvfBtYygplX

`y_train`: https://drive.google.com/open?id=1hv24Ufiio9rBeSqgnNoM3dr5sVGwOmBy

`x_test`: https://drive.google.com/open?id=1AH9lKHT5P2oQLz8SGMRPWs_M9wIM2ZRH

`y_test`: https://drive.google.com/open?id=1i4_azocSDuU3TcDf3OSHO1vF0D5-xMU6

You should store these files in Drive for use in this Colab notebook.

In [ ]:
# Run this cell to connect to your Drive folder

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Import the Eurosat data from your Drive folder

def load_eurosat_data():
    x_train = np.load('/content/gdrive/MyDrive/x_train.npy')
    y_train = np.load('/content/gdrive/MyDrive/y_train.npy')
    x_test  = np.load('/content/gdrive/MyDrive/x_test.npy')
    y_test  = np.load('/content/gdrive/MyDrive/y_test.npy')
    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_eurosat_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

#### Build the neural network model

You can now construct a model to fit to the data. Using the Sequential API, build your model according to the following specifications:

* The model should use the input_shape in the function argument to set the input size in the first layer.
* The first layer should be a Conv2D layer with 16 filters, a 3x3 kernel size, a ReLU activation function and 'SAME' padding. Name this layer 'conv_1'.
* The second layer should also be a Conv2D layer with 8 filters, a 3x3 kernel size, a ReLU activation function and 'SAME' padding. Name this layer 'conv_2'.
* The third layer should be a MaxPooling2D layer with a pooling window size of 8x8. Name this layer 'pool_1'.
* The fourth layer should be a Flatten layer, named 'flatten'.
* The fifth layer should be a Dense layer with 32 units, a ReLU activation. Name this layer 'dense_1'.
* The sixth and final layer should be a Dense layer with 10 units and softmax activation. Name this layer 'dense_2'.

In total, the network should have 6 layers.

In [ ]:
#### GRADED CELL ####

# Complete the following function. 
# Make sure to not change the function name or arguments.

def get_new_model(input_shape):
  model = Sequential ([
      Conv2D(filters=16, kernel_size=(3,3), input_shape=input_shape, padding='same', activation='relu', name='conv_1'),
      Conv2D(filters=8, kernel_size=(3,3), activation='relu', padding='same', name='conv_2'),
      MaxPooling2D((8, 8), name='pool_1'),
      Flatten(name='flatten'),
      Dense(32, activation='relu', name='dense_1'),
      Dense(10, activation='softmax', name='dense_2')
  ])

  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

#### Compile and evaluate the model

In [ ]:
# Run your function to create the model

model = get_new_model(x_train[0].shape)

In [ ]:
# Run this cell to define a function to evaluate a model's test accuracy

def get_test_accuracy(model, x_test, y_test):
    """Test model classification accuracy"""
    test_loss, test_acc = model.evaluate(x=x_test, y=y_test, verbose=0)
    print('accuracy: {acc:0.3f}'.format(acc=test_acc))

In [ ]:
# Print the model summary and calculate its initialised test accuracy

model.summary()
get_test_accuracy(model, x_test, y_test)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 conv_2 (Conv2D)             (None, 64, 64, 8)         1160      
                                                                 
 pool_1 (MaxPooling2D)       (None, 8, 8, 8)           0         
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                16416     
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
Total params: 18,354
Trainable params: 18,354
Non-trai

#### Create checkpoints to save model during training, with a criterion

You will now create three callbacks:
- `checkpoint_every_epoch`: checkpoint that saves the model weights every epoch during training
- `checkpoint_best_only`: checkpoint that saves only the weights with the highest validation accuracy. Use the testing data as the validation data.
- `early_stopping`: early stopping object that ends training if the validation accuracy has not improved in 3 epochs.

In [ ]:
#### GRADED CELL ####

# Complete the following functions. 
# Make sure to not change the function names or arguments.

def get_checkpoint_every_epoch():
  path ='checkpoints_every_epoch/chechpoint_{epoch:03d}'
  checking = ModelCheckpoint(filepath =path, save_weights_only=True, frequency='epoch', verbose=1)
  return checking

def get_checkpoint_best_only():
  path2='checkpoints_best_only/checkpoint'
  modelcheck = ModelCheckpoint(filepath=path2, save_best_olny=True, monitor='val_accuracy', mode='max', verbose=1)
  return modelcheck 

In [ ]:
#### GRADED CELL ####

# Complete the following function. 
# Make sure to not change the function name or arguments.

def get_early_stopping():
  stopping = EarlyStopping(monitor='val_accuracy', patience=3, mode='auto')
  return stopping   

In [ ]:
# Run this cell to create the callbacks

checkpoint_every_epoch = get_checkpoint_every_epoch()
checkpoint_best_only = get_checkpoint_best_only()
early_stopping = get_early_stopping()

#### Train model using the callbacks

Now, you will train the model using the three callbacks you created. If you created the callbacks correctly, three things should happen:
- At the end of every epoch, the model weights are saved into a directory called `checkpoints_every_epoch`
- At the end of every epoch, the model weights are saved into a directory called `checkpoints_best_only` **only** if those weights lead to the highest test accuracy
- Training stops when the testing accuracy has not improved in three epochs.

You should then have two directories:
- A directory called `checkpoints_every_epoch` containing filenames that include `checkpoint_001`, `checkpoint_002`, etc with the `001`, `002` corresponding to the epoch
- A directory called `checkpoints_best_only` containing filenames that include `checkpoint`, which contain only the weights leading to the highest testing accuracy

In [ ]:
# Train model using the callbacks you just created

callbacks = [checkpoint_every_epoch, checkpoint_best_only, early_stopping]
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=callbacks)

Epoch 1/50
125/125 [==============================] - ETA: 0s - loss: 2.1170 - accuracy: 0.1865
Epoch 1: saving model to checkpoints_every_epoch/chechpoint_001

Epoch 1: saving model to checkpoints_best_only/checkpoint


125/125 [==============================] - 14s 108ms/step - loss: 2.1170 - accuracy: 0.1865 - val_loss: 1.8069 - val_accuracy: 0.2990
Epoch 2/50
125/125 [==============================] - ETA: 0s - loss: 1.5347 - accuracy: 0.4180
Epoch 2: saving model to checkpoints_every_epoch/chechpoint_002

Epoch 2: saving model to checkpoints_best_only/checkpoint


125/125 [==============================] - 14s 111ms/step - loss: 1.5347 - accuracy: 0.4180 - val_loss: 1.4372 - val_accuracy: 0.4140
Epoch 3/50
125/125 [==============================] - ETA: 0s - loss: 1.3565 - accuracy: 0.4622
Epoch 3: saving model to checkpoints_every_epoch/chechpoint_003

Epoch 3: saving model to checkpoints_best_only/checkpoint


125/125 [==============================] - 13s 106ms/step - loss: 1.3565 - accuracy: 0.4622 - val_loss: 1.3419 - val_accuracy: 0.4600
Epoch 4/50
125/125 [==============================] - ETA: 0s - loss: 1.2565 - accuracy: 0.5090
Epoch 4: saving model to checkpoints_every_epoch/chechpoint_004

Epoch 4: saving model to checkpoints_best_only/checkpoint


125/125 [==============================] - 13s 105ms/step - loss: 1.2565 - accuracy: 0.5090 - val_loss: 1.3063 - val_accuracy: 0.4860
Epoch 5/50
125/125 [==============================] - ETA: 0s - loss: 1.2055 - accuracy: 0.5460
Epoch 5: saving model to checkpoints_every_epoch/chechpoint_005

Epoch 5: saving model to checkpoints_best_only/checkpoint


...

125/125 [==============================] - 14s 114ms/step - loss: 0.5196 - accuracy: 0.8148 - val_loss: 0.7790 - val_accuracy: 0.7440
Epoch 31/50
125/125 [==============================] - ETA: 0s - loss: 0.5062 - accuracy: 0.8217
Epoch 31: saving model to checkpoints_every_epoch/chechpoint_031

Epoch 31: saving model to checkpoints_best_only/checkpoint


125/125 [==============================] - 13s 107ms/step - loss: 0.5062 - accuracy: 0.8217 - val_loss: 0.7404 - val_accuracy: 0.7530
Epoch 32/50
125/125 [==============================] - ETA: 0s - loss: 0.4816 - accuracy: 0.8280
Epoch 32: saving model to checkpoints_every_epoch/chechpoint_032

Epoch 32: saving model to checkpoints_best_only/checkpoint


125/125 [==============================] - 13s 105ms/step - loss: 0.4816 - accuracy: 0.8280 - val_loss: 0.7782 - val_accuracy: 0.7290
Epoch 33/50
125/125 [==============================] - ETA: 0s - loss: 0.4886 - accuracy: 0.8290
Epoch 33: saving model to checkpoints_every_epoch/chechpoint_033

Epoch 33: saving model to checkpoints_best_only/checkpoint


125/125 [==============================] - 14s 110ms/step - loss: 0.4886 - accuracy: 0.8290 - val_loss: 0.7910 - val_accuracy: 0.7290
Epoch 34/50
125/125 [==============================] - ETA: 0s - loss: 0.4773 - accuracy: 0.8353
Epoch 34: saving model to checkpoints_every_epoch/chechpoint_034

Epoch 34: saving model to checkpoints_best_only/checkpoint


125/125 [==============================] - 14s 108ms/step - loss: 0.4773 - accuracy: 0.8353 - val_loss: 0.7978 - val_accuracy: 0.7260


#### Create new instance of model and load on both sets of weights

Now you will use the weights you just saved in a fresh model. You should create two functions, both of which take a freshly instantiated model instance:
- `model_last_epoch` should contain the weights from the latest saved epoch
- `model_best_epoch` should contain the weights from the saved epoch with the highest testing accuracy

_Hint: use the_ `tf.train.latest_checkpoint` _function to get the filename of the latest saved checkpoint file. Check the docs_ [_here_](https://www.tensorflow.org/api_docs/python/tf/train/latest_checkpoint).

In [ ]:
#### GRADED CELL ####

# Complete the following functions. 
# Make sure to not change the function name or arguments.

def get_model_last_epoch(model):
  last_epochfile = tf.train.latest_checkpoint(checkpoint_dir='checkpoints_every_epoch')
  model.load_weights(last_epochfile)
  return model

def get_model_best_epoch(model):
  model.load_weights('checkpoints_best_only/checkpoint')
  return model

In [ ]:
# Run this cell to create two models: one with the weights from the last training
# epoch, and one wiht the weights leading to the highest validation (testing) accuracy.
# Verify that the second has a higher validation (testing) accuarcy.

model_last_epoch = get_model_last_epoch(get_new_model(x_train[0].shape))
model_best_epoch = get_model_best_epoch(get_new_model(x_train[0].shape))
print('Model with last epoch weights:')
get_test_accuracy(model_last_epoch, x_test, y_test)
print('')
print('Model with best epoch weights:')
get_test_accuracy(model_best_epoch, x_test, y_test)

Model with last epoch weights:
accuracy: 0.726

Model with best epoch weights:
accuracy: 0.726


#### Load, from scratch, a model trained on the EuroSat dataset.

In your workspace, you will find another model trained on the `EuroSAT` dataset in `.h5` format. This model is trained on a larger subset of the EuroSAT dataset and has a more complex architecture. The path to the model is `models/EuroSatNet.h5`. See how its testing accuracy compares to your model!

In [ ]:
#### GRADED CELL ####

# Complete the following functions. 
# Make sure to not change the function name or arguments.

def get_model_eurosatnet():
  model= load_model('/content/gdrive/MyDrive/EuroSatNet.h5')
  return model
    
    

In [ ]:
# Run this cell to print a summary of the EuroSatNet model, along with its validation accuracy.

model_eurosatnet = get_model_eurosatnet()
model_eurosatnet.summary()
get_test_accuracy(model_eurosatnet, x_test, y_test)

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 conv_2 (Conv2D)             (None, 64, 64, 16)        6416      
                                                                 
 pool_1 (MaxPooling2D)       (None, 32, 32, 16)        0         
                                                                 
 conv_3 (Conv2D)             (None, 32, 32, 16)        2320      
                                                                 
 conv_4 (Conv2D)             (None, 32, 32, 16)        6416      
                                                                 
 pool_2 (MaxPooling2D)       (None, 16, 16, 16)        0         
                                                                 
 conv_5 (Conv2D)             (None, 16, 16, 16)      

Congratulations for completing this programming assignment! You're now ready to move on to the capstone project for this course.